In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 500)
import tabula
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import tukey_hsd
import pingouin as pg

In [2]:
movie_data_v1 = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\semester1\elements_of_data_processing\assignment2\movie_data_intactversion.csv")

In [3]:
# take three types of age certifications (R, NC-17, TV-Y7) of movie
movie_data_3age = movie_data_v1[movie_data_v1['age_certification'].isin(['R', 'NC-17', 'TV-Y7'])].reset_index().drop(['index'], axis = 1).iloc[:, :6]
movie_data_3age = movie_data_3age[movie_data_3age['tmdb_popularity'].isna() == False].reset_index().drop(['index'], axis = 1)
movie_data_3age['tmdb_popularity_adj'] = np.log2(movie_data_3age['tmdb_popularity'])
movie_data_3age['genres'] = movie_data_3age['genres'].astype('string')

In [4]:
# see the freqeucy for genres
movie_data_3age_v1 = movie_data_3age.copy()
movie_data_3age_v1['genres'] = movie_data_3age_v1['genres'].apply(lambda x : eval(x))
genres_freq_dict = {}
for genre_lst in list(movie_data_3age_v1['genres']):
    for genre in genre_lst:
        if genres_freq_dict.get(genre) == None:
            genres_freq_dict[genre] = 1
        else:
            genres_freq_dict[genre] += 1

In [5]:
# drama, action, thriller, and comedy
genres_freq_dict

{'drama': 366,
 'crime': 177,
 'action': 256,
 'thriller': 277,
 'european': 70,
 'comedy': 262,
 'romance': 78,
 'documentation': 62,
 'war': 37,
 'music': 28,
 'horror': 86,
 'scifi': 111,
 'animation': 109,
 'fantasy': 95,
 'family': 89,
 'western': 16,
 'history': 29,
 'sport': 19,
 'reality': 2}

In [6]:
# see the number of instances for each indicated age certification
movie_data_3age['age_certification'].value_counts()

R        556
TV-Y7    119
NC-17     16
Name: age_certification, dtype: int64

In [7]:
movie_data_3age['genres'] = movie_data_3age['genres'].apply(lambda x : str(x))
movie_data_3age['genres'] = movie_data_3age['genres'].apply(lambda x : x.replace('[', '').replace(']', ''))
movie_data_3age = movie_data_3age[movie_data_3age['genres'] != ''].reset_index().drop(['index'], axis = 1)

In [8]:
movie_R_drama = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'R') & ((movie_data_3age['genres'].str.contains('drama')))].reset_index().drop(['index'], axis = 1)
movie_R_action = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'R') & ((movie_data_3age['genres'].str.contains('action')))].reset_index().drop(['index'], axis = 1)
movie_R_thriller = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'R') & ((movie_data_3age['genres'].str.contains('thriller')))].reset_index().drop(['index'], axis = 1)
movie_R_comedy = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'R') & ((movie_data_3age['genres'].str.contains('comedy')))].reset_index().drop(['index'], axis = 1)
print(len(movie_R_drama), len(movie_R_action), len(movie_R_thriller), len(movie_R_comedy))

movie_NC_drama = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'NC-17') & ((movie_data_3age['genres'].str.contains('drama')))].reset_index().drop(['index'], axis = 1)
movie_NC_action = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'NC-17') & ((movie_data_3age['genres'].str.contains('action')))].reset_index().drop(['index'], axis = 1)
movie_NC_thriller = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'NC-17') & ((movie_data_3age['genres'].str.contains('thriller')))].reset_index().drop(['index'], axis = 1)
movie_NC_comedy = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'NC-17') & ((movie_data_3age['genres'].str.contains('comedy')))].reset_index().drop(['index'], axis = 1)
print(len(movie_NC_drama), len(movie_NC_action), len(movie_NC_thriller), len(movie_NC_comedy))

movie_Y7_drama = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'TV-Y7') & ((movie_data_3age['genres'].str.contains('drama')))].reset_index().drop(['index'], axis = 1)
movie_Y7_action = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'TV-Y7') & ((movie_data_3age['genres'].str.contains('action')))].reset_index().drop(['index'], axis = 1)
movie_Y7_thriller = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'TV-Y7') & ((movie_data_3age['genres'].str.contains('thriller')))].reset_index().drop(['index'], axis = 1)
movie_Y7_comedy = movie_data_3age.loc[(movie_data_3age['age_certification'] == 'TV-Y7') & ((movie_data_3age['genres'].str.contains('comedy')))].reset_index().drop(['index'], axis = 1)
print(len(movie_Y7_drama), len(movie_Y7_action), len(movie_Y7_thriller), len(movie_Y7_comedy))

322 169 262 169
12 9 7 4
32 78 8 89


In [9]:
def age_popu(df):
    return np.array(df['tmdb_popularity_adj'])
drama_R = age_popu(movie_R_drama)
action_R = age_popu(movie_R_action)
thriller_R = age_popu(movie_R_thriller)
comedy_R = age_popu(movie_R_comedy)

drama_NC = age_popu(movie_NC_drama)
action_NC = age_popu(movie_NC_action)
thriller_NC = age_popu(movie_NC_thriller)
comedy_NC = age_popu(movie_NC_comedy)

drama_Y7 = age_popu(movie_Y7_drama)
action_Y7 = age_popu(movie_Y7_action)
thriller_Y7 = age_popu(movie_Y7_thriller)
comedy_Y7 = age_popu(movie_Y7_comedy)

In [10]:
f_statistic, p_value = stats.f_oneway(drama_R, action_R, thriller_R, comedy_R, drama_NC, action_NC, thriller_NC, comedy_NC, 
                                     drama_Y7, action_Y7, thriller_Y7, comedy_Y7)
print('F-test outcomes: p-value = {}'.format(p_value))

F-test outcomes: p-value = 4.202208311212737e-07


In [11]:
tukey_results = tukey_hsd(drama_R, action_R, thriller_R, comedy_R, drama_NC, action_NC, thriller_NC, comedy_NC, 
                                     drama_Y7, action_Y7, thriller_Y7, comedy_Y7)

In [16]:
print(drama_R.mean(), action_R.mean(), thriller_R.mean(), comedy_R.mean())
print(drama_NC.mean(), action_NC.mean(), thriller_NC.mean(), comedy_NC.mean())
print(drama_Y7.mean(), action_Y7.mean(), thriller_Y7.mean(), comedy_Y7.mean())

4.190541445995166 4.571680670590645 4.395676338983441 3.6017443663656428
4.480844591855159 3.935305221277528 3.667050437766228 1.9147268112353841
4.34581230137588 3.9015557356042585 3.7290223611500437 3.9046586993899983


In [13]:
# 0-3, 1-3, 1-7, 2-3
print(tukey_results)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)     -0.381     0.339    -0.880     0.117
 (0 - 2)     -0.205     0.930    -0.642     0.232
 (0 - 3)      0.589     0.006     0.090     1.087
 (0 - 4)     -0.290     1.000    -1.833     1.253
 (0 - 5)      0.255     1.000    -1.519     2.029
 (0 - 6)      0.523     0.999    -1.482     2.529
 (0 - 7)      2.276     0.172    -0.365     4.916
 (0 - 8)     -0.155     1.000    -1.128     0.818
 (0 - 9)      0.289     0.958    -0.373     0.951
 (0 - 10)      0.462     1.000    -1.417     2.340
 (0 - 11)      0.286     0.943    -0.343     0.914
 (1 - 0)      0.381     0.339    -0.117     0.880
 (1 - 2)      0.176     0.994    -0.342     0.694
 (1 - 3)      0.970     0.000     0.399     1.541
 (1 - 4)      0.091     1.000    -1.477     1.659
 (1 - 5)      0.636     0.992    -1.159     2.432
 (1 - 6)      0.905     0.950    -1.120     2.929
 (1 - 7)      2.657     0.050 